# Intsall YOLOv8

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 7.4 MB/s eta 0:00:00


# Easy OCR

Install Easy OCR

In [3]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.9 MB/s eta 0:00:00


Importing EasyOCR

In [4]:
import easyocr
reader = easyocr.Reader(['en'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Detecting License Plate and Recognizing Text using EasyOCR

In [15]:
from ultralytics import YOLO
import cv2
import os
import csv
import easyocr

# Initialize YOLO model
model = YOLO('best1.pt')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Path to the folder containing input images
input_folder = 'images'

# Path to save the CSV file
csv_file_path = 'easyocr.csv'

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to CSV file
    csv_writer.writerow(['Image', 'Text', 'Score'])

    # Iterate through each image in the folder
    for image_file in os.listdir(input_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, image_file)

            # Perform inference on the image
            results = model(image_path)

            # Load the original image
            img = cv2.imread(image_path)

            # Initialize a list to store recognized license numbers for the current image
            recognized_license_numbers = []

            # Extract bounding boxes
            boxes = results[0].boxes.xyxy.tolist()

            # Iterate through the bounding boxes
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = box
                # Crop the object using the bounding box coordinates
                cropped_object = img[int(y1):int(y2), int(x1):int(x2)]
                # Perform OCR on the cropped object
                detections = reader.readtext(cropped_object)
                #output_image_path = f'{image_file}crop{i}.png'
                # cv2.imwrite(output_image_path, cropped_object)
                # Iterate through the OCR detections
                for detection in detections:
                    bbox, text, score = detection

                    # Convert text to uppercase and remove spaces
                    text = text.upper().replace(' ', '')

                    # Append recognized license number to the list
                    recognized_license_numbers.append(text)

                    # Print the result to the console
                    print(f"Image: {image_file}, Text: {text}, Score: {score}")


            # Insert the complete recognized license numbers for the current image into the CSV file
            csv_writer.writerow([image_file, ''.join(recognized_license_numbers), ''])

print("Results saved to", csv_file_path)


image 1/1 /content/images/car-wbs-MH14EH7958_00001_jpeg.rf.1bf2a9f7ea04cb2342791e4e27109d3d.jpg: 288x640 1 license_plate, 24.6ms
Speed: 1.7ms preprocess, 24.6ms inference, 2.9ms postprocess per image at shape (1, 3, 288, 640)
Image: car-wbs-MH14EH7958_00001_jpeg.rf.1bf2a9f7ea04cb2342791e4e27109d3d.jpg, Text: HHH14EH79587, Score: 0.238781695558064

image 1/1 /content/images/20231116_121738.jpg: 640x480 1 license_plate, 16.9ms
Speed: 4.2ms preprocess, 16.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)
Image: 20231116_121738.jpg, Text: HONDA, Score: 0.9085428236311156
Image: 20231116_121738.jpg, Text: IND, Score: 0.999906191412006
Image: 20231116_121738.jpg, Text: UP70G, Score: 0.37103498259528855
Image: 20231116_121738.jpg, Text: A3329, Score: 0.7300174405188765

image 1/1 /content/images/94c5a151-24b5-493c-a900-017a4353b00c___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Tata-Tiago-Front-Number-Plates-Design_jpg.rf.211f69e546fc959be29af247e00b7f8d.jpg: 480x640 1 license_pla

# Pytesseract OCR

Install Pytesseract OCR

In [12]:
!apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 9 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,208 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

Detecting License Plate and Recognizing Text using Pytesseract-OCR

In [13]:
from ultralytics import YOLO
import cv2
import pytesseract
import os
import csv

# Load the YOLOv8 model
model = YOLO('best1.pt')

# Path to the folder containing input images
input_folder = 'images'

# Path to save the CSV file
csv_file_path = 'pytesseract.csv'

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to CSV file
    csv_writer.writerow(['Image', 'Bounding Box', 'Text', 'Confidence Score'])

    # Iterate through each image in the folder
    for image_file in os.listdir(input_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, image_file)

            # Perform inference on the image
            results = model(image_path)

            # Load the original image
            img = cv2.imread(image_path)

            # Extract bounding boxes
            boxes = results[0].boxes.xyxy.tolist()

            # Iterate through the bounding boxes
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = box
                # crop license plate
                license_plate_crop = img[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                grayscale_resize_test_license_plate = cv2.cvtColor( license_plate_crop, cv2.COLOR_BGR2GRAY)
                gaussian_blur_license_plate = cv2.GaussianBlur(grayscale_resize_test_license_plate, (5, 5), 0)
                # read license plate number
                custom_config = r'--oem 3 -l eng --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                data = pytesseract.image_to_data(gaussian_blur_license_plate, lang='eng', config=custom_config, output_type=pytesseract.Output.DICT)
                complete_text = ""
                bounding_box = None
                confidence_score = 0  # Initialize to a low value
                output_image_path = f'{image_file}crop{i}.png'
                # cv2.imwrite(output_image_path, license_plate_crop)
                # Iterate through the data to access individual text elements and their properties
                for i in range(len(data['text'])):
                    text = data['text'][i].strip()
                    confidence = int(data['conf'][i])
                    if text:
                        complete_text += text + ' '  # Add recognized text to complete_text
                        confidence_score = max(confidence_score, confidence)

                        # Update the bounding box to include the entire recognized text
                        if bounding_box is None:
                            x, y, w, h = int(data['left'][i]), int(data['top'][i]), int(data['width'][i]), int(data['height'][i])
                            bounding_box = [x, y, x + w, y + h]
                        else:
                            x, y, w, h = int(data['left'][i]), int(data['top'][i]), int(data['width'][i]), int(data['height'][i])
                            left, top, right, bottom = bounding_box
                            bounding_box = [min(left, x), min(top, y), max(right, x + w), max(bottom, y + h)]

                # Write the result to the CSV file
                csv_writer.writerow([image_file, bounding_box, complete_text.strip(), confidence_score])

print("Results saved to", csv_file_path)


image 1/1 /content/images/car-wbs-MH14EH7958_00001_jpeg.rf.1bf2a9f7ea04cb2342791e4e27109d3d.jpg: 288x640 1 license_plate, 37.7ms
Speed: 1.9ms preprocess, 37.7ms inference, 7.3ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/images/20231116_121738.jpg: 640x480 1 license_plate, 73.3ms
Speed: 4.2ms preprocess, 73.3ms inference, 9.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/images/94c5a151-24b5-493c-a900-017a4353b00c___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Tata-Tiago-Front-Number-Plates-Design_jpg.rf.211f69e546fc959be29af247e00b7f8d.jpg: 480x640 1 license_plate, 59.2ms
Speed: 9.1ms preprocess, 59.2ms inference, 15.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/images/car-wbs-TN21AT0480_00001_jpg.rf.d298dae137b6a785e0d18bdaef1a9b21.jpg: 576x640 1 license_plate, 20.8ms
Speed: 4.0ms preprocess, 20.8ms inference, 1.8ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /content/images/car-wbs-TN07BU5427_00000_jp

# Keras-OCR

Install Keras-OCR

In [9]:
!pip install -q keras-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00


Detecting License Plate and Recognizing Text using EasyOCR

In [16]:
import cv2
import keras_ocr
import numpy as np
import os
import csv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best1.pt')

# Path to the folder containing input images
input_folder = 'images'

# Path to save the CSV file
csv_file_path = 'kerasocr.csv'

# Initialize the Keras OCR detector and recognizer
pipeline = keras_ocr.pipeline.Pipeline()

# List to store recognized license plate numbers
recognized_license_plates = []

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to CSV file
    csv_writer.writerow(['Image', 'Text', 'Confidence Score', 'Bounding Box'])

    # Iterate through each image in the folder
    for image_file in os.listdir(input_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, image_file)

            # Perform inference on the image
            results = model(image_path)

            # Load the original image
            img = cv2.imread(image_path)

            # Get the bounding boxes from YOLO detection
            boxes = results[0].boxes.xyxy.tolist()

            # Iterate through the bounding boxes
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                # Crop license plate
                license_plate_crop = img[y1:y2, x1:x2]

                max_width = 800

                # Calculate the aspect ratio and resize accordingly
                aspect_ratio = license_plate_crop.shape[1] / float(license_plate_crop.shape[0])
                new_width = min(max_width, license_plate_crop.shape[1])
                new_height = int(new_width / aspect_ratio)

                # Resize the image
                resized_image = cv2.resize(license_plate_crop, (new_width, new_height))

                # Continue with the preprocessing steps...

                # Convert the image to grayscale
                gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

                # Apply Gaussian blur to reduce noise and enhance text
                blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

                # Perform adaptive thresholding to create a binary image
                thresh_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

                # Apply morphological operations to clean up the image
                kernel = np.ones((3, 3), np.uint8)
                processed_image = cv2.morphologyEx(thresh_image, cv2.MORPH_CLOSE, kernel)

                # Invert the image to have white text on a black background
                inverted_image = cv2.bitwise_not(processed_image)

                images = [processed_image]
                prediction_groups = pipeline.recognize(images)

                # Extract the recognized text and confidence
                recognized_text = ""
                confidence_scores = []

                for predictions in prediction_groups:
                    for text, confidence in predictions:
                        recognized_text += text + ' '
                        confidence_scores.append(confidence)

                max_confidence = np.max(confidence_scores) if confidence_scores else 0.0
                # Store the recognized license plate and its bounding box
                recognized_license_plates.append({
                    'Image': image_file,
                    'Text': recognized_text.strip(),
                    'Confidence Score': max_confidence,
                    'Bounding Box': (x1, y1, x2, y2)
                })
                csv_writer.writerow([image_file, recognized_text.strip(), max_confidence, (x1, y1, x2, y2)])

# Write results to CSV file

print("Results saved to", csv_file_path)


Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5

image 1/1 /content/images/car-wbs-MH14EH7958_00001_jpeg.rf.1bf2a9f7ea04cb2342791e4e27109d3d.jpg: 288x640 1 license_plate, 18.9ms
Speed: 1.5ms preprocess, 18.9ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)
1/1 [==============================] - 3s 3s/step

image 1/1 /content/images/20231116_121738.jpg: 640x480 1 license_plate, 12.2ms
Speed: 3.0ms preprocess, 12.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
1/1 [==============================] - 1s 570ms/step

image 1/1 /content/images/94c5a151-24b5-493c-a900-017a4353b00c___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Tata-Tiago-Front-Number-Plates-Design_jpg.rf.211f69e546fc959be29af247e00b7f8d.jpg: 480x640 1 license_plate, 12.1ms
Speed: 2.5ms preprocess, 12.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 1s 509ms/step

image 1/1 /content/images/car-wbs-TN21AT0480_00001_jpg.rf.d298dae137b6a785e0d18

# Automatic Number Plate Recognistion in Real-Time

In [18]:
!yolo task=detect mode=predict model=best1.pt conf=0.55 source=sample1.mp4

Ultralytics YOLOv8.0.215 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (1/451) /content/sample1.mp4: 384x640 (no detections), 99.3ms
video 1/1 (2/451) /content/sample1.mp4: 384x640 (no detections), 10.1ms
video 1/1 (3/451) /content/sample1.mp4: 384x640 (no detections), 10.5ms
video 1/1 (4/451) /content/sample1.mp4: 384x640 (no detections), 9.8ms
video 1/1 (5/451) /content/sample1.mp4: 384x640 (no detections), 8.1ms
video 1/1 (6/451) /content/sample1.mp4: 384x640 (no detections), 8.7ms
video 1/1 (7/451) /content/sample1.mp4: 384x640 (no detections), 11.6ms
video 1/1 (8/451) /content/sample1.mp4: 384x640 (no detections), 9.9ms
video 1/1 (9/451) /content/sample1.mp4: 384x640 (no detections), 9.4ms
video 1/1 (10/451) /content/sample1.mp4: 384x640 (no detections), 7.5ms
video 1/1 (11/451) /content/sample1.mp4: 384x640 (no detections), 9.9ms
video 1/1 (12/451) /content/sample1.mp4: 38

In [20]:
def license_complies_format(text):
    if len(text) <= 5:
        return False
    else:
        return True

In [21]:
from ultralytics import YOLO
import cv2
cap = cv2.VideoCapture('sample1.mp4')
results = {}
model = YOLO('best1.pt')


frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:

        license = model(frame)[0]
        boxes = license.boxes.xyxy.tolist()

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box
            # Crop the object using the bounding box coordinates
            license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
            # Convert the cropped object to grayscale
            license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
            # Apply thresholding
            _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)
            detections = reader.readtext(license_plate_crop)
            for detection in detections:
              bbox, text, score = detection
              if license_complies_format(text):
                results[frame_nmr]= {'license_plate': {'bbox': [x1, y1, x2, y2],
                                                 'text': text,
                                                 'text_score': score}}
csv_file = 'test.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['frame_nmr', 'x1', 'y1', 'x2', 'y2', 'text', 'text_score'])

    for frame_nmr, info in results.items():
        license_plate_info = info['license_plate']
        writer.writerow([frame_nmr] + license_plate_info['bbox'] + [license_plate_info['text'], license_plate_info['text_score']])


0: 384x640 (no detections), 10.8ms
Speed: 2.1ms preprocess, 10.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.2ms
Speed: 2.9ms preprocess, 14.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.0ms
Speed: 2.8ms preprocess, 15.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 2.8ms preprocess, 11.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.8ms
Speed: 3.0ms preprocess, 13.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 3.3ms preprocess, 12.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.4ms
Speed: 3.6ms preprocess, 15.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.1ms
Speed: 3.5ms preprocess, 13.1ms i